# Práctica 1 - CART Splitting
El objetivo de esta práctica consiste en implemetar el algoritmo de CART Splitting, el cuál se basa en iterar por las variables del conjunto de entrada de forma que se minimice la siguiente expresión:

![Figura 1. Expresión a minimizar](./img/expresion.png)

## Algoritmo
El algoritmo para minimizar esta suma es el siguiente:
1. Elegimos un valor de X1 que no sea ni el primero ni el último. Ese valor determinará las regiones (s).
2. Aplicamos la suma con ese valor de X1.
3. Repetimos para todos los valores de X1.
4. El valor que nos dé la suma mínima será el que elegiremos.

## Código
Primero, cargamos el dataframe que vamos a usar para crear el árbol

In [3]:
library(dplyr)
df <- read.csv("./datasets/datos.csv")
recid_iva <- read.csv("./datasets/datosicb.txt", sep = " ",
                      colClasses = c("recid" = "factor", "grado" = "factor",
                                     "feno" = "factor", "quim" = "factor",
                                     "horm" = "factor"))
recid_iva <- recid_iva %>%
  mutate(recid = sort(as.numeric(recid)), edad = sort(edad),
         tam = sort(tam), gang = sort(gang), grado = sort(as.numeric(grado)),
         feno = sort(as.numeric(feno)), quim = sort(as.numeric(quim)),
         horm = sort(as.numeric(horm)))
cart_tree <- list(root = df, left = NA, right = NA)
df$x1 <- sort(df$x1)
df
recid_iva

x1,x2,y
<dbl>,<dbl>,<int>
1.728571,1.7847839,0
2.771245,1.1697614,0
2.999209,2.8128136,0
3.678320,2.6199503,0
3.961043,2.2090142,0
6.642287,3.1629535,1
7.444542,3.3390472,1
7.497546,0.4766834,1
9.002203,3.2345510,1


edad,tam,grado,gang,feno,quim,horm,recid
<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
25,0.2,1,0,1,1,1,1
27,0.3,1,0,1,1,1,1
28,0.3,1,0,1,1,1,1
29,0.4,1,0,1,1,1,1
30,0.5,1,0,1,1,1,1
30,0.5,1,0,1,1,1,1
31,0.5,1,0,1,1,1,1
31,0.6,1,0,1,1,1,1
31,0.6,1,0,1,1,1,1


Posteriormente, creamos la función que implementa el algoritmo en cuestión.

In [4]:
cart_splitting <- function(data, class_variable,
                           verbose = FALSE) {
  data_size <- dim(data)[1]
  min_global_sum <- Inf
  vars <- names(data[, unlist(lapply(data, is.numeric))])
  best_var <- NA
  best_s <- Inf
  vars_size <- length(vars)
  for (var in vars[1:vars_size - 1]) {
    min_sum <- Inf
    aux_data <- data[order(data[[var]]), ]
    for (i in 2:(data_size - 1)) {
      aux_s <- aux_data[[var]][i]
      y_vector_r1 <- aux_data[[class_variable]][aux_data[[var]] < aux_s]
      y_vector_r2 <- aux_data[[class_variable]][aux_data[[var]] >= aux_s]
      aux_sum <- sum(y_vector_r1 - mean(y_vector_r1)^2)
      + sum(y_vector_r2 - mean(y_vector_r2)^2)

      if (aux_sum <= min_sum) {
        min_sum <- aux_sum
        s <- aux_s
      }
      if (verbose) {
        print(sprintf("Variable: %f; Valor: %f; Suma mínima: %f", aux_s,
                      aux_data[[class_variable]][aux_data[[var]] == aux_s],
                      aux_sum))
      }
    }
    if (min_sum <= min_global_sum) {
      min_global_sum <- min_sum
      best_s <- s
      best_var <- var
    }
  }
  r1 <- data[data[[best_var]] < best_s, ]
  r2 <- data[data[[best_var]] >= best_s, ]
  data <- data[order(data[[best_var]]), ]
  return(list(R1 = r1, R2 = r2, value = best_s,
              best_var = best_var, min_sum = min_global_sum))
}

Una vez encontrado el valor que garantiza la suma mínima, obtenemos de la lista resultado las regiones, la suma mínima y el valor de *s*.

In [5]:
split_df <- cart_splitting(df, "y")
split_recid_iva <- cart_splitting(recid_iva, "recid")
split_df
split_recid_iva

,x1,x2,y
,<dbl>,<dbl>,<int>
8,7.497546,0.4766834,1
,x1,x2,y
,<dbl>,<dbl>,<int>
1,1.728571,1.784784,0
2,2.771245,1.169761,0
3,2.999209,2.812814,0
4,3.678320,2.619950,0
5,3.961043,2.209014,0
6,6.642287,3.162954,1


,edad,tam,grado,gang,feno,quim,horm,recid
,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,25,0.2,1,0,1,1,1,1
2,27,0.3,1,0,1,1,1,1
3,28,0.3,1,0,1,1,1,1
4,29,0.4,1,0,1,1,1,1
5,30,0.5,1,0,1,1,1,1
6,30,0.5,1,0,1,1,1,1
7,31,0.5,1,0,1,1,1,1
8,31,0.6,1,0,1,1,1,1
9,31,0.6,1,0,1,1,1,1
